In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
#from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:

drive.mount('/content/drive')

In [ ]:
# Load the data
train_data_EV = pd.read_csv('Train.csv')
test_data_EV = pd.read_csv('Test.csv')

# Inspect the data to understand its structure
print(train_data_EV.head())
#print(test_data_EV.head())

Pre-Processing

In [ ]:
# Normalize the feature
scaler = MinMaxScaler()
train_data_EV['VehicleSpeed_km_h_'] = scaler.fit_transform(train_data_EV[['VehicleSpeed_km_h_']])
test_data_EV['VehicleSpeed_km_h_'] = scaler.transform(test_data_EV[['VehicleSpeed_km_h_']])


Prepare the Input and Target Variables


In [ ]:
# Separate features (X) and target (y)
X_train = train_data_EV[['VehicleSpeed_km_h_']].values
y_train = train_data_EV['Battery_Status'].values

X_test = test_data_EV[['VehicleSpeed_km_h_']].values
y_test = test_data_EV['Battery_Status'].values

# Reshape X to have 3D shape: (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

Build the RNN Model

In [ ]:
# Create the RNN model
model = Sequential()
model.add(SimpleRNN(50, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

# Compile the model
model.compile(optimizer='RMSprop', loss='binary_crossentropy',metrics=['accuracy'] )

Train The Model

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

Predictions

In [ ]:
# Get predictions on the test set
y_pred = model.predict(X_test)

# Convert probabilities to binary classes (0 or 1)
y_pred_classes = (y_pred > 0.5).astype(int)

# Use y_pred_classes in classification_report
report = classification_report(y_test, y_pred_classes, output_dict=True)
report_df = pd.DataFrame(report).transpose()

Generate the Report

In [ ]:
report = classification_report(y_test, y_pred_classes)
print("Classification Report:\n")
print(report)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(report_df.iloc[:-1, :-1], annot=True, cmap='Blues', fmt=".2f")
plt.title('Classification Report')
plt.ylabel('Metrics')
plt.xlabel('Classes')
plt.show()

Calculate AUC Score: Compute the AUC using roc_auc_score.



In [ ]:
y_prob = model.predict(X_test)
auc_score = roc_auc_score(y_test, y_prob)
print(f"AUC Score: {auc_score:.4f}")


Plot the ROC Curve: Generate the ROC curve and plot it.



In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'RNN Model (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()